In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_density, plot_line, get_post_df
import altair as alt
from codebase.classes_data import Data
from codebase.ibis import exp_and_normalise
from run_ibis_lvm2 import run_ibis_lvm
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace
from copy import copy


alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
log_dir = 'log/20210129_202956_testsmc2//'
exp_data = load_obj('data', log_dir)

##

In [3]:
particles = load_obj('particles', log_dir)
particles.particles[5].particles['beta']

array([[0.03884088],
       [0.91070133],
       [0.31099934],
       [0.04376871],
       [0.4131238 ],
       [1.36347754]])

In [4]:
particles.particles[0].particles

OrderedDict([('beta', array([[-0.47993332],
                     [ 0.29562908],
                     [-0.67341458],
                     [-0.77232687],
                     [-1.1118089 ],
                     [-0.62929958]])),
             ('alpha',
              array([-0.5651791 ,  0.21698791, -0.99183803, -1.35618763, -1.28303828,
                     -1.90121949]))])

In [5]:
particles.resample_particles_bundles()

In [6]:
ps = dict()
samples = np.empty((particles.size, 1, 6))
for i in range(particles.size):
    samples[i] = particles.particles[i].particles['alpha']
ps['alpha'] = samples
samples = np.empty((particles.size, 6, 1))
for i in range(particles.size):
    samples[i] = particles.particles[i].particles['beta']
ps['beta'] = samples

## Or load existing directory

In [7]:
# log_dir = '/log/20210122_173348_mcmc_ibis_adapt9/'
# ps = load_obj( 'mcmc_post_samples', log_dir)
# ps['beta'] = ps['beta'][num_warmup:].copy()

In [8]:
ps['beta'].shape

(200, 6, 1)

## Post process loadings for sign flips

In [9]:
nsim = ps['beta'].shape[0]
nrows = ps['beta'].shape[1]
for n in range(nsim):
    for i in range(nrows):
        sign = np.sign(ps['beta'][n,0])
        ps['beta'][n] = sign * ps['beta'][n,]

## Plot MCMC samples

In [10]:
param = 'beta'
df = get_post_df(ps[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.Chart(...)

In [11]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2', 'source']]

dd = pd.DataFrame(exp_data.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [12]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)